<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#XGBoost" data-toc-modified-id="XGBoost-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a href="https://xgboost.readthedocs.io/en/latest/index.html" target="_blank">XGBoost</a></a></span><ul class="toc-item"><li><span><a href="#Baseline-Model" data-toc-modified-id="Baseline-Model-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Baseline Model</a></span><ul class="toc-item"><li><span><a href="#Prepare-Data" data-toc-modified-id="Prepare-Data-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Prepare Data</a></span></li><li><span><a href="#Train" data-toc-modified-id="Train-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Train</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#Average-Treatment-Effect" data-toc-modified-id="Average-Treatment-Effect-1.1.3.1"><span class="toc-item-num">1.1.3.1&nbsp;&nbsp;</span>Average Treatment Effect</a></span></li></ul></li></ul></li><li><span><a href="#New-Model" data-toc-modified-id="New-Model-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>New Model</a></span><ul class="toc-item"><li><span><a href="#Prepare-Data" data-toc-modified-id="Prepare-Data-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Prepare Data</a></span><ul class="toc-item"><li><span><a href="#Categorical-Features" data-toc-modified-id="Categorical-Features-1.2.1.1"><span class="toc-item-num">1.2.1.1&nbsp;&nbsp;</span>Categorical Features</a></span></li><li><span><a href="#Pipeline" data-toc-modified-id="Pipeline-1.2.1.2"><span class="toc-item-num">1.2.1.2&nbsp;&nbsp;</span>Pipeline</a></span></li></ul></li><li><span><a href="#Train" data-toc-modified-id="Train-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Train</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Evaluation</a></span></li></ul></li></ul></li><li><span><a href="#Features-Importance" data-toc-modified-id="Features-Importance-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Features Importance</a></span><ul class="toc-item"><li><span><a href="#SHAP" data-toc-modified-id="SHAP-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://github.com/slundberg/shap" target="_blank">SHAP</a></a></span></li><li><span><a href="#XGBoost-features-importance" data-toc-modified-id="XGBoost-features-importance-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span><a href="https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.Booster.get_score" target="_blank">XGBoost features importance</a></a></span></li></ul></li><li><span><a href="#Practical-Lessons-From-Facebook" data-toc-modified-id="Practical-Lessons-From-Facebook-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://quinonero.net/Publications/predicting-clicks-facebook.pdf" target="_blank">Practical Lessons From Facebook</a></a></span><ul class="toc-item"><li><span><a href="#Prepare-Data" data-toc-modified-id="Prepare-Data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Prepare Data</a></span></li><li><span><a href="#Train" data-toc-modified-id="Train-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Train</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#Make-a-Submission" data-toc-modified-id="Make-a-Submission-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Make a Submission</a></span></li></ul></li></ul></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

Продолжаем работать над задачей CTR-prediction с использованием датасета от Criteo.

Описание задачи и данных можно посмотреть в notebook'e предыдущей практики (`sgd_logreg_nn/notebooks/ctr_prediction_mllib.ipynb`).

# [XGBoost](https://xgboost.readthedocs.io/en/latest/index.html)


Утановим xgboost

In [2]:
!pip3.5 install xgboost

In [3]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
from typing import Dict
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


sys.path.append('./utils')

os.environ['PYSPARK_SUBMIT_ARGS'] = """
--jars xgboost4j-spark-0.72.jar,xgboost4j-0.72.jar
--py-files sparkxgb.zip pyspark-shell
""".replace('\n', ' ')

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

from metrics import rocauc, logloss, ne, calibration
from processing import split_by_col

from sparkxgb.xgboost import *

Обратите внимание на слудующие строки:

* ```python
sys.path.append('./utils')
...
from metrics import rocauc, logloss, ne
from processing import split_by_col
```

В папке `utils` находится два файла (`metrics.py`, `processing.py`), которые содержат функции, которые нужно было реализовать в рамках предыдущей практики.


* ```python
os.environ['PYSPARK_SUBMIT_ARGS'] = """
--jars xgboost4j-spark-0.72.jar,xgboost4j-0.72.jar
--py-files sparkxgb.zip pyspark-shell
"""
...
from sparkxgb.xgboost import *
```

Для того чтобы в рамках инфраструктуры Spark можно было использовать XGBoost, мы воспользуемся библиотекой [XGBoost4J](https://xgboost.readthedocs.io/en/latest/jvm/xgboost4j_spark_tutorial.html).

В ходе выполнения занятий может быть полезно ознакомиться с исходным кодом обертки для питона, который находится в архиве `sparkxgb.zip`

In [4]:
DATA_PATH = '/workspace/data/criteo/dac'
TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

MODEL_DIR = '/workspace/models/criteo/dac'

In [5]:
TRAIN_SAMPLING_RATE = 0.1
SEED=82

In [6]:
all_metrics = {}

## Baseline Model
### Prepare Data

In [7]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

In [8]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [9]:
df = df.fillna(0, subset=num_columns)

Воспользуемся пайплайном из предыдущей практики

In [10]:
from pyspark.ml import PipelineModel


pipeline_model = PipelineModel.load(os.path.join(DATA_PATH, 'pipeline_model'))

In [11]:
pipeline_model.stages

[StringIndexer_ffdd8d524b01,
 StringIndexer_547ed9cdbe45,
 OneHotEncoderEstimator_b3d576ca1390,
 VectorAssembler_9c298301fb70]

In [12]:
len(pipeline_model.stages[0].labels), len(pipeline_model.stages[1].labels)

(1389, 552)

Итоговая размерность пространства фичей

In [13]:
dim = len(num_columns) + len(pipeline_model.stages[0].labels) + len(pipeline_model.stages[1].labels)
dim

1954

In [14]:
df = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

df.count()

3667985

In [ ]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

Exception ignored in: <function JavaObject.__init__.<locals>.<lambda> at 0x7fd23aab8e18>
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1293, in <lambda>
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 625, in _garbage_collect_object
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 920, in garbage_collect_object
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 983, in send_command
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 931, in _get_connection
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 937, in _create_connection
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1071, in start
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1087, in _authenticate_connection
  

In [ ]:
train_df = train_df.sample(False, TRAIN_SAMPLING_RATE, seed=SEED)

### Train

In [ ]:
estimator = XGBoostEstimator(
    featuresCol="features", 
    labelCol="label", 
    predictionCol="prediction",
    colsample_bytree=0.9,
    eta=0.15,
    gamma=0.9,
    max_depth=8,
    min_child_weight=50.0,
    subsample=0.9,
    objective='binary:logistic',
    eval_metric='logloss', 
    silent=0,
    num_round=20,
    nthread=1,
    nworkers=1
)

In [ ]:
model = estimator.fit(train_df)

Сохраняем [booster](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.Booster) обученной модели

In [ ]:
XGB_BASELINE_PATH = os.path.join(MODEL_DIR, 'xgb.model')

model._call_java("booster").saveModel(XGB_BASELINE_PATH)

### Evaluation

#### Average Treatment Effect

Пусть даны две экспериментальные группы treatment ($T$) и control ($C$), где

* `treatment` - группа с изменением (например, новая модель)
* `control` - группа без изменений

Рассмотрим метрику $X$, значение которой мы расчитали для наших групп ($X_T, X_C$).

Тогда под ATE будем иметь в виду
$$ \Delta\% = \frac{X_T - X_C}{X_C} \cdot 100 $$

In [ ]:
def get_ate(groups, control_name) -> pd.DataFrame:
    """Get Average Treatment Effect
    groups - dictionary where keys - names of models, values - dicts of pairs <metric_name>, <metric_value>
    control_name - name of baseline model
    
    return pd.DataFrame (rows corresponds to metrics, cols corresponds to models and ATE with respect to control)
    """
    
    metric_names = []
    for metric_name_values in groups.values():
        for metric_name, _ in metric_name_values.items():
            if metric_name not in metric_names:
                metric_names.append(metric_name)
    metric_names = list(sorted(metric_names))
    
    if control_name not in groups:
        raise ValueError("Control experiment is not in the group.")
    control_values = groups[control_name]
    if len(control_values) != len(metric_names):
        raise ValueError("Control experiment does not have all the metrics computed.")

    model_names = list(sorted(groups.keys()))
    metric_model_ates = []
    for metric_name in metric_names:
        control_value = control_values[metric_name]
        model_ates = []
        for model_name in model_names:
            if metric_name in groups[model_name]:
                ate = (groups[model_name][metric_name] - control_value) / control_value * 100
            else:
                ate = None
            model_ates.append(ate)
        metric_model_ates.append(model_ates)

    ates_df = pd.DataFrame(data=metric_model_ates, index=metric_names, columns=model_names)
    return ates_df

In [ ]:
ROC_AUC_HANDLE = 'Area Under ROC'
CALIBRATION_HANDLE = 'Calibration'

In [ ]:
baseline_metrics = {}
baseline_metrics[ROC_AUC_HANDLE] = rocauc(model, test_df, probabilities_col='probabilities')
baseline_metrics[CALIBRATION_HANDLE] = calibration(model, test_df, probabilities_col='probabilities')
baseline_metrics

In [ ]:
XGB_BASELINE_HANDLE = '1.xgb_baseline'
all_metrics[XGB_BASELINE_HANDLE] = baseline_metrics

Сравним результаты с логрег моделью из предыдущей практики.

1. Загрузить обученную `LogReg` модель
2. Посчитать метрики на `test_df`

In [ ]:
from pyspark.ml.classification import LogisticRegressionModel


LOG_REG_MODEL_PATH = os.path.join(MODEL_DIR, 'log_reg_model')

log_reg_model = LogisticRegressionModel.load(LOG_REG_MODEL_PATH)

In [ ]:
log_reg_metrics = {}
log_reg_metrics[ROC_AUC_HANDLE] = rocauc(log_reg_model, test_df)
log_reg_metrics[CALIBRATION_HANDLE] = calibration(log_reg_model, test_df)
log_reg_metrics

In [ ]:
LOG_REG_HANDLE = '2.logistic_regression'
all_metrics[LOG_REG_HANDLE] = log_reg_metrics

Построить таблицу ATE используя метод `get_ate`

In [ ]:
get_ate(all_metrics, XGB_BASELINE_HANDLE)

## New Model
### Prepare Data

In [ ]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)
df = df.sample(False, SAMPLE_RATE, seed=SEED)
df = df.fillna(0, subset=num_columns)

#### Categorical Features

See [Doc](https://spark.apache.org/docs/latest/ml-pipeline.html) for additional details on Transformers and Encoders.

Implement classes below

In [ ]:
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param, Params
from pyspark import keyword_only


class MeanTargetEncoderModel(
    pyspark.ml.Model,
    HasInputCol,
    HasOutputCol,
    pyspark.ml.util.DefaultParamsWritable,
    pyspark.ml.util.DefaultParamsReadable):
    """Fitted Model"""
    
    def __init__(self):        
        super(MeanTargetEncoderModel, self).__init__()
        self.inputOutputMapping = Param(self, "inputOutputMapping", "inputOutputMapping")
        self._setDefault(inputOutputMapping={})

    @keyword_only
    def setParams(self, inputCol, outputCol, inputOutputMapping):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def setInputOutputMapping(self, value):
        return self._set(inputOutputMapping=value)

    def getInputOutputMapping(self):
        return self.getOrDefault(self.inputOutputMapping)
        
    def transform(self, dataset: pyspark.sql.DataFrame) -> pyspark.sql.DataFrame:
        inputOutputMapping = self.getInputOutputMapping()
        def map_column_using_mapping(input_value, default_value=0.0):
            return inputOutputMapping.get(input_value, default_value)
    
        map_column_using_mapping_udf = F.udf(map_column_using_mapping, FloatType())

        return dataset \
            .withColumn(self.getOutputCol(), map_column_using_mapping_udf(self.getInputCol()))


class MeanTargetEncoder(pyspark.ml.Estimator):
    """Estimator"""

    def __init__(self, inputCol: str, featuresCol: str, outputCol: str,):
        self.inputCol = inputCol
        self.featuresCol = featuresCol
        self.outputCol = outputCol
    
    def fit(self, dataset: pyspark.sql.DataFrame) -> MeanTargetEncoderModel:
        inputOutputMapping = dataset \
            .groupby(self.inputCol) \
            .agg(F.mean(self.featuresCol).alias(self.outputCol)) \
            .select(F.col(self.inputCol), F.col(self.outputCol)) \
            .rdd \
            .collectAsMap()
        mte_model = MeanTargetEncoderModel()
        mte_model.setParams(
            inputCol=self.inputCol, outputCol=self.outputCol, inputOutputMapping=inputOutputMapping)
        return mte_model

In [ ]:
cat_enc_columns = [cat_col + '_enc' for cat_col in cat_columns]

cat_col, cat_enc_col = cat_columns[0], cat_enc_columns[0]
mean_target_encoder = MeanTargetEncoder(cat_col, '_c0', cat_enc_col)
mean_target_encoder_model = mean_target_encoder.fit(df)

In [ ]:
mean_target_encoder_model.transform(df).take(1)

#### Pipeline

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler


cat_enc_columns = [cat_col + '_enc' for cat_col in cat_columns]

mean_target_encoders = [MeanTargetEncoder(cat_col, '_c0', cat_enc_col) 
                        for cat_col, cat_enc_col in zip(cat_columns, cat_enc_columns)]

assembler = VectorAssembler(inputCols=num_columns + cat_enc_columns, outputCol="features").setHandleInvalid("keep")

pipeline = Pipeline(stages=mean_target_encoders + [assembler])

In [ ]:
MTE_PIPELINE_MODEL_PATH = os.path.join(DATA_PATH, 'pipeline_model_2')

pipeline_model = pipeline.fit(df)
pipeline_model.write().overwrite().save(MTE_PIPELINE_MODEL_PATH)

In [ ]:
from pyspark.ml import PipelineModel


pipeline_model = PipelineModel.load(MTE_PIPELINE_MODEL_PATH)
pipeline_model.stages

In [ ]:
df = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

df.count()

In [ ]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

In [ ]:
train_df.take(1)

In [ ]:
dim = len(train_df.take(1)[0].features)
dim

### Train

Train XGBoost on the new set of features

In [ ]:
estimator = XGBoostEstimator(
    featuresCol="features", 
    labelCol="label", 
    predictionCol="prediction",
    colsample_bytree=0.9,
    eta=0.15,
    gamma=0.9,
    max_depth=8,
    min_child_weight=50.0,
    subsample=0.9,
    objective='binary:logistic',
    eval_metric='logloss', 
    silent=0,
    num_round=20,
    nthread=1,
    nworkers=1
)

In [ ]:
model = estimator.fit(train_df)

In [ ]:
XGB_MTE_PATH = os.path.join(MODEL_DIR, 'xgb.model_2')

model._call_java("booster").saveModel(XGB_MTE_PATH)

### Evaluation

Сравнить результаты новой модели с `xgb_baseline` и `log_reg` с помощью функции `get_ate`

In [ ]:
xgb_mte_metrics = {}
xgb_mte_metrics[ROC_AUC_HANDLE] = rocauc(model, test_df, probabilities_col='probabilities')
xgb_mte_metrics[CALIBRATION_HANDLE] = calibration(model, test_df, probabilities_col='probabilities')
xgb_mte_metrics

In [ ]:
XGB_MTE_HANDLE = '3.xgb_mte'

all_metrics[XGB_MTE_HANDLE] = xgb_mte_metrics

In [ ]:
get_ate(all_metrics, XGB_BASELINE_HANDLE)

# Features Importance

## [SHAP](https://github.com/slundberg/shap)

Install SHAP

In [ ]:
!pip3.5 install shap

In [ ]:
import shap
import xgboost as xgb

# load JS visualization code to notebook
shap.initjs()

bst = xgb.Booster()
bst.load_model(XGB_MTE_PATH)

Имея `booster` модели можно, например, посмотреть на то какие деревья получились в итоге.

In [ ]:
print(bst.get_dump(dump_format="text")[0])

In [ ]:
sample_df = df.sample(False, 0.05, seed=SEED)
sample_df.count()

In [ ]:
from pyspark.ml.linalg import DenseVector, SparseVector, _convert_to_vector
from scipy.sparse import csr_matrix


def df_to_csr(df, dim):
    data = []
    row_ind = []
    col_ind = []
    
    vecs = df.rdd.map(lambda row: row.features).collect()
    for i, vec in enumerate(vecs):
        if isinstance(vec, SparseVector):
            indices = vec.indices
        elif isinstance(vec, DenseVector):
            indices = range(len(vec))
        else:
            raise NotImplementedValue()
        
        for idx, val in zip(indices, vec.values):
            data.append(val)
            row_ind.append(i)
            col_ind.append(idx)
        
    return csr_matrix((data, (row_ind, col_ind)), shape=(len(vecs), dim))

In [ ]:
%%time
X = df_to_csr(sample_df, dim)
dtest = xgb.DMatrix(X)

In [ ]:
X_arr = X.toarray()

In [ ]:
%%time
explainer = shap.TreeExplainer(bst)
shap_values = explainer.shap_values(dtest, tree_limit=20)

In [ ]:
shap.summary_plot(shap_values, X_arr, max_display=20, alpha=0.3)

## [XGBoost features importance](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.Booster.get_score)

In [ ]:
def get_feature_score(booster, importance):
    gains_xgb = booster.get_score(importance_type=importance)
    gains = {}
    for f, g in gains_xgb.items():
        gains[f] = g
    sorted_gains = sorted(list(gains.items()), key=lambda x: -x[1])
    return sorted_gains


features_scores = get_feature_score(bst, 'gain')[:20]

In [ ]:
import pandas as pd


f_names, f_scores = zip(*features_scores)
features_scores_pdf = pd.DataFrame({'feature': f_names, 'gain': f_scores})

In [ ]:
import seaborn as sns


_, ax = plt.subplots(figsize=(8,8))
ax = sns.barplot(x='gain', y='feature', data=features_scores_pdf)
plt.show()

# [Practical Lessons From Facebook](https://quinonero.net/Publications/predicting-clicks-facebook.pdf)

## Prepare Data

* Реализуйте модель из статьи (LogReg поверх XGBoost)

* Попробуйте реализовать Negatives Subsampling + Re-calibration описанный в статье (доп. баллы)

In [ ]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH) \
    .fillna(0, subset=num_columns)
df = df.sample(False, SAMPLE_RATE, seed=SEED)

In [ ]:
pipeline_model = PipelineModel.load(MTE_PIPELINE_MODEL_PATH)
pipeline_model.stages

In [ ]:
df = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

df.count()

In [ ]:
def add_tree_leaves_to_row(row, has_label=True):
    result = [row.id]
    if has_label:
        result.append(row.label)

    samples = np.array([row.features.toArray()])
    xgb_matrix = xgb.DMatrix(samples)
    bst = xgb.Booster()
    bst.load_model(XGB_MTE_PATH)
    leaves = bst.predict(xgb_matrix, pred_leaf=True)[0]
    for leaf in leaves:
        result.append(int(leaf))
    return result

bst = xgb.Booster()
bst.load_model(XGB_MTE_PATH)
num_trees = len(bst.get_dump())
leaf_columns = ["leaf" + str(leaf_num) for leaf_num in range(num_trees)]

df = df \
    .rdd \
    .map(lambda row: add_tree_leaves_to_row(row, has_label=True)) \
    .toDF(["id", "label"] + leaf_columns) \
    .cache()

df.head()

In [ ]:
from pyspark.ml.feature import OneHotEncoderEstimator

leaf_encoded_columns = [leaf_col + '_enc' for leaf_col in leaf_columns]

leaf_encoder = OneHotEncoderEstimator(inputCols=leaf_columns, outputCols=leaf_encoded_columns)

assembler = VectorAssembler(inputCols=leaf_encoded_columns, outputCol="features").setHandleInvalid("keep")

pipeline = Pipeline(stages=[leaf_encoder, assembler])

In [ ]:
TREE_PIPELINE_MODEL_PATH = os.path.join(DATA_PATH, 'pipeline_model_3')

pipeline_model = pipeline.fit(df)
pipeline_model.write().overwrite().save(TREE_PIPELINE_MODEL_PATH)

In [ ]:
pipeline_model = PipelineModel.load(TREE_PIPELINE_MODEL_PATH)
pipeline_model.stages

In [ ]:
df = pipeline_model \
    .transform(df) \
    .select('label', 'features', 'id') \
    .cache()

df.head()

In [ ]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

## Train

In [ ]:
from pyspark.ml.classification import LogisticRegression

def find_the_best_log_reg_model(train_df, val_df, negative_downsampling_rate):
    best_model = None
    best_rocauc = None
    for regParam in [1e-1]:  #  [0.0, 1e-1, 1e-2, 1e-3]:
        for elasticNetParam in [1e-2]:  # [0.0, 1e-1, 1e-2, 1e-3]:
            for standardization in [True]:
                lr = LogisticRegression(
                    regParam=regParam, elasticNetParam=elasticNetParam, standardization=standardization,
                    probabilityCol='probabilities')
                lr_model = lr.fit(train_df)
                lr_rocauc = rocauc(
                    lr_model,
                    val_df,
                    probabilities_col='probabilities',
                    negative_downsampling_rate=negative_downsampling_rate)
                if best_rocauc is None or best_rocauc < lr_rocauc:
                    print("regParam={} elasticNetParam={} standardization={}".format(
                        regParam, elasticNetParam, standardization))
                    print("Improved ROCAUC from {} to {} with the params.".format(best_rocauc, lr_rocauc))
                    best_model = lr_model
                    best_rocauc = lr_rocauc
    return best_model, best_rocauc

## Negative subsampling and recalibration

Although it is not the best way to do downsampling and comparing it with the performance with on the 100% dataset, we will still do it. To do a fair comparison, we should have compared the equal amounts of samples, i.e. 10% of all data downsampled uniformly vs. all postivie + the negative downsampling rate which would bring the total number of samples equal to 10% of all the dataset.

The inconvenience is in the dataset preprocessing and the structure of the code above which compares all previous models on a 10% test set of a 50% uniformly downsampled dataset.

The final model for the contest will do model selection by first splitting the whole set and then operating only on the train set.

In [ ]:
train_df.count()

In [ ]:
positive_train_df = train_df.filter(F.col('label') == 1.0)
positive_train_df.count()

In [ ]:
negative_train_df = train_df.filter(F.col('label') == 0.0)
negative_train_df.count()

In [ ]:
best_model, best_rocauc, best_neg_rate = None, None, None
for segment in range(1, 21):
    negative_downsampling_rate = segment * 0.05
    print('*' * 100)
    print('NEGATIVE DOWNSAMPLING RATE =', negative_downsampling_rate)
    rebalanced_train_df = negative_train_df \
        .sample(False, negative_downsampling_rate, seed=SEED) \
        .union(positive_train_df)
    cur_model, cur_rocauc = find_the_best_log_reg_model(
        rebalanced_train_df,
        val_df,
        negative_downsampling_rate)
    if best_rocauc is None or best_rocauc < cur_rocauc:
        print('Imporved the global best ROCAUC model!')
        best_model = cur_model
        best_rocauc = cur_rocauc
        best_neg_rate = negative_downsampling_rate

In [ ]:
best_rocauc, best_neg_rate

In [ ]:
LOG_REG_XGB_MODEL_PATH = os.path.join(MODEL_DIR, 'log_reg_xgb_model_' + str(best_neg_rate))

best_model.write().overwrite().save(LOG_REG_XGB_MODEL_PATH)

## Evaluation

In [ ]:
# calibration(model, df) is implemented in the utils.metrics.

Сравните новую модель со всеми предыдущими с помощью `get_ate`. При сравнении использовать еще и метрику calibration

In [ ]:
logistic_regression_xgb_metrics = {}
logistic_regression_xgb_metrics[ROC_AUC_HANDLE] = rocauc(
    best_model, test_df, probabilities_col='probabilities', negative_downsampling_rate=best_neg_rate)
logistic_regression_xgb_metrics[CALIBRATION_HANDLE] = calibration(
    best_model, test_df, probabilities_col='probabilities', negative_downsampling_rate=best_neg_rate)
logistic_regression_xgb_metrics

In [ ]:
LOG_REG_XGB_MODEL_HANDLE = '4.logistic_regression_xgb'
all_metrics[LOG_REG_XGB_MODEL_HANDLE] = logistic_regression_xgb_metrics

In [ ]:
get_ate(all_metrics, XGB_BASELINE_HANDLE)

### Make a Submission

Если в результате работы получилась модель, которая лучше чем ЛогРег из предыдущей практики, то точно нужно сделать submission.

In [ ]:
TEST_DATA_PATH = '/workspace/data/mlbd-20-ctr-prediction-1'
TEST_TEST_PATH = os.path.join(TEST_DATA_PATH, 'test.csv')

comp_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TEST_TEST_PATH) \
    .fillna(0, subset=num_columns)

comp_df.count()

In [ ]:
pipeline_model = PipelineModel.load(MTE_PIPELINE_MODEL_PATH)
pipeline_model.stages

In [ ]:
comp_df = pipeline_model \
    .transform(comp_df) \
    .select('features', 'id') \
    .cache()

In [ ]:
comp_df = comp_df \
    .rdd \
    .map(lambda row: add_tree_leaves_to_row(row, has_label=False)) \
    .toDF(["id"] + leaf_columns) \
    .cache()

In [ ]:
pipeline_model = PipelineModel.load(TREE_PIPELINE_MODEL_PATH)
pipeline_model.stages

In [ ]:
comp_df = pipeline_model \
    .transform(comp_df) \
    .select('features', 'id') \
    .cache()

In [ ]:
comp_df_prediction = best_model.transform(comp_df)

In [ ]:
comp_df_prediction.take(1)

In [ ]:
comp_predictions = comp_df_prediction \
    .rdd \
    .map(lambda row: (row.id, row.probabilities[1])) \
    .collect()

In [ ]:
submission_path = os.path.join(TEST_DATA_PATH, 'submission_logistic_regression_xgb.csv')
with open(submission_path, 'w') as writer:
    writer.write("id,proba\n")
    for row in comp_predictions:
        writer.write(",".join(map(str, row))+"\n")